In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from pathlib import Path

In [36]:
import string as str

In [212]:
# Information about attendees of all events 
data_events_ppl = pd.read_csv("../raw_data/raw_all.csv")
data_events_ppl.sample(15)

,Event,First Name,Surname,Email,Attendee Status,Your Job Position,Choose your role,Choose your role.1,Seniority,How did you hear from us?,Company,Rain,Day,Location
74,2,Soumitra,Sahu,soumitra.sahu89@gmail.com,Not Attending,NaN,NaN,NaN,NaN,NaN,Tourlane,Yes,Tuesday,Berlin
789,5,Marcin,Belinski,4rz27gdtzp@privaterelay.appleid.com,Checked In,Product,Product Manager,NaN,NaN,I know the organizers,Element Insurance AG,No,Thursday,Berlin
459,3,Dominik,Hess,mail.dominikhess@gmail.com,Checked In,Product,NaN,Product Manager,NaN,Lenny's Newsletter,Bundesdruckerei GmbH,Yes,Thursday,Berlin
628,5,Yuri,Orthey,yehtro@gmail.com,Not Attending,NaN,NaN,NaN,NaN,NaN,-,No,Thursday,Berlin
100,2,Muhammad Banirahma,Syahroni,email@bani.us,Not Attending,NaN,NaN,NaN,NaN,NaN,Delivery Hero,Yes,Tuesday,Berlin
339,1,Elif,Özçakmak,ozcakmakelif@gmail.com,Checked In,NaN,NaN,NaN,NaN,NaN,NaN,No,Thursday,Berlin
64,2,Mathieu,Ritter,mathieu@mr-recruitment.com,Not Attending,NaN,NaN,NaN,NaN,NaN,MR Recruitment,Yes,Tuesday,Berlin
151,2,Dan,Krause,mrdanleonkrause@gmail.com,Checked In,Design,NaN,NaN,NaN,LinkedIn,DLK,Yes,Tuesday,Berlin
27,2,Anmol,Kawatra,anmolkawatra796@gmail.com,Attending,Founder / CXO,NaN,NaN,NaN,Word of Mouth / Referral,Climanu,Yes,Tuesday,Berlin
607,5,Mritunjay Kumar,Verma,mritunjaykumarv@gmail.com,Attending,Product,Product Manager,NaN,NaN,LinkedIn,.,No,Thursday,Berlin


In [140]:
data_events_ppl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Event                      1067 non-null   int64 
 1   First Name                 1067 non-null   object
 2   Surname                    1067 non-null   object
 3   Email                      1067 non-null   object
 4   Attendee Status            1067 non-null   object
 5   Your Job Position          616 non-null    object
 6   Choose your role           433 non-null    object
 7   Choose your role.1         79 non-null     object
 8   Seniority                  62 non-null     object
 9   How did you hear from us?  676 non-null    object
 10  Company                    994 non-null    object
 11  Rain                       1067 non-null   object
 12  Day                        1067 non-null   object
 13  Location                   1067 non-null   object
dtypes: int64

In [213]:
# ever updating file with data scraped from linked in
data_scraped = pd.read_csv("../raw_data/raw_scrapped.csv")
data_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625 entries, 0 to 624
Data columns (total 69 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   url                        612 non-null    object 
 1   title                      612 non-null    object 
 2   linkedinProfileUrl         564 non-null    object 
 3   email                      162 non-null    object 
 4   linkedinProfile            564 non-null    object 
 5   description                427 non-null    object 
 6   headline                   600 non-null    object 
 7   location                   600 non-null    object 
 8   firstName                  600 non-null    object 
 9   lastName                   600 non-null    object 
 10  fullName                   600 non-null    object 
 11  connectionDegree           600 non-null    object 
 12  vmid                       600 non-null    object 
 13  userId                     600 non-null    float64

In [215]:


data_events_ppl["First Name"] = data_events_ppl["First Name"].apply(process_name)
data_events_ppl["Surname"] = data_events_ppl["Surname"].apply(process_name)
data_events_ppl["fullName"] = data_events_ppl["First Name"] + ' ' + data_events_ppl["Surname"]

data_events_ppl['Email'] = data_events_ppl['Email'].fillna('').str.lower()
data_events_ppl['Company'] = data_events_ppl["Company"].apply(process_name)

# Create DF with unique IDs------------------------------

# Define custom function to get the first non-null value
def first_non_null(series):
    return series.dropna().iloc[0] if not series.dropna().empty else np.nan

# Columns where you want to apply the first_non_null function
columns_to_agg = ['First Name',
                  'Surname',
                  'Email',
                  'Company',
                  'Your Job Position',
                  'Choose your role',
                  'Choose your role.1',
                  'Seniority'
                  ]

# Create a dictionary to specify aggregation functions for each column

def calculate_attended(group):

    # Get the last event for the group
    last_event = group.iloc[-1]

    #breakpoint()

    # Check if the last event's Attendee Status is "Checked In"
    if last_event == 'Checked In':
        return 1
    else:
        return 0


agg_dict = {'Event': 'count', 'Attendee Status': calculate_attended }
agg_dict.update({column: first_non_null for column in columns_to_agg})
# Group by UserID and apply the calculate_attended function

# Group by 'fullName' and apply aggregation
unique_attendees = data_events_ppl.groupby(by='fullName', as_index=False).agg(agg_dict)
unique_attendees["numEvents"] = unique_attendees.Event

# Index in this table is now the user ID
unique_attendees['UserID'] = unique_attendees.index

# we have numEvents now instead
unique_attendees = unique_attendees.drop(labels='Event',axis=1)
unique_attendees["Company"] = unique_attendees["Company"].apply(process_name)

# Clean data_scraped
data_scraped.lastName = data_scraped.lastName.apply(process_name)
data_scraped.firstName = data_scraped.firstName.apply(process_name)
data_scraped.fullName = data_scraped.firstName + ' ' + data_scraped.lastName
data_scraped.dropna(subset=['url'])

data_scraped['email'] = data_scraped['email'].fillna('').str.lower()
data_scraped['company'] = data_scraped['company'].fillna('').str.lower().str.strip()

data_scraped['UserID'] = float('nan')
data_scraped['Attendance'] = 0


In [216]:
data_scraped

,url,title,linkedinProfileUrl,email,linkedinProfile,description,headline,location,firstName,lastName,...,imgUrl,website,mail,connectedOn,error,phoneNumber,partialScreenshot,facebookUrl,UserID,Attendance
0,https://linkedin.com/in/patrick-beeker-4673311,Patrick Beeker,https://www.linkedin.com/in/patrick-beeker-467...,,https://www.linkedin.com/in/patrick-beeker-467...,Experienced professional with the ability to c...,"General Manager - US Finance, Legal, HR, Accou...","Carrollton, Texas, United States",patrick,beeker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,https://linkedin.com/in/teena-kumari-0b8359271,Teena Kumari – Tech product support intern – H...,https://www.linkedin.com/in/teena-kumari-0b835...,,https://www.linkedin.com/in/teena-kumari-0b835...,"I am a Product Owner at World Simplified UG, a...",Tech product support intern at Holidu,"Berlin, Berlin, Germany",teena,kumari,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,https://linkedin.com/in/mischagawronski,Mischa Gawronski – Creative Director – Freelance,https://www.linkedin.com/in/mischagawronski/,,https://www.linkedin.com/in/mischagawronski/,Passionate and challenging influencer in strat...,IGP Network | Creative Director,"Berlin, Berlin, Germany",mischa,gawronski,...,https://media.licdn.com/dms/image/C5603AQHb9Mw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,https://linkedin.com/in/shuvam-kumar-sah-88590...,Shuvam Kumar Sah – Web Designer,https://www.linkedin.com/in/shuvam-kumar-sah-8...,,https://www.linkedin.com/in/shuvam-kumar-sah-8...,Designer & Problem Solver | CS Student | Aimin...,Making Interfaces Sexier - UI/UX/Dev,"Berlin, Berlin, Germany",shuvam,sah,...,https://media.licdn.com/dms/image/D4E03AQFcJ48...,dribbble.com/shuvam_sah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,https://linkedin.com/in/chalhoub,Ayman Chalhoub - Clink,https://www.linkedin.com/in/chalhoub/,,https://www.linkedin.com/in/chalhoub/,NaN,Product & Growth,"Berlin, Berlin, Germany",ayman,chalhoub,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,https://linkedin.com/in/nadaelnadi/,Nada El Nadi - Junior Product Designer - Velents,https://www.linkedin.com/in/nadaelnadi/,,https://www.linkedin.com/in/nadaelnadi/,A lady with a previous marketing background an...,Junior Product Designer✨| UX/UI | Figma | Rese...,"Berlin, Berlin, Germany",nada,nadi,...,https://media.licdn.com/dms/image/D4E03AQHYHGw...,nadaelnadi.super.site/,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,NaN,0
621,https://linkedin.com/in/advabendov,Adva BenDov - Spryker,https://www.linkedin.com/in/advabendov/,,https://www.linkedin.com/in/advabendov/,"A passionate Product Leader Focused on People,...",Product Leader & Mentor • Lead Product Manager...,"Berlin, Berlin, Germany",adva,bendov,...,https://media.licdn.com/dms/image/C4D03AQEni61...,NaN,NaN,"Feb 26, 2024",NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,NaN,0
622,https://linkedin.com/in/georgianaciobotaru,Georgiana Ciobotaru - HelloFresh,https://www.linkedin.com/in/georgianaciobotaru/,,https://www.linkedin.com/in/georgianaciobotaru/,"Product Management, Data & Technology enthusia...",Product @HelloFresh,"Berlin, Berlin, Germany",georgiana,ciobotaru,...,https://media.licdn.com/dms/image/C4D03AQFZ2yJ...,NaN,NaN,"Jun 18, 2019",NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,NaN,0
623,https://linkedin.com/in/valentinklein,Valentin Klein – HelloFresh,https://www.linkedin.com/in/valentinklein/,,https://www.linkedin.com/in/valentinklein/,Emerging Product Leader with 8+ years of exper...,Building New Ventures | Product @HelloFresh | ...,"Dubai, United Arab Emirates",valentin,klein,...,https://media.licdn.com/dms/image/D4E03AQFZ93Q...,calendly.com/valentin-klein-hf/30min,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,NaN,0


In [218]:
# Iterate through each row in the ids DataFrame
for j, row_ids in unique_attendees.iterrows():
    # Iterate through each row in the scraped DataFrame
    for index_scraped, row_scraped in data_scraped.iterrows():
        # Check if the rows match using the custom matching function
        if custom_user_matching(row_scraped, row_ids):
            # If a match is found, add the userID from the ids DataFrame to the scraped DataFrame
            data_scraped.at[index_scraped, 'UserID'] = row_ids['UserID']
            data_scraped.at[index_scraped, 'Attendance'] = row_ids['Attendee Status']
            break
# Drop rows with NaN values in the 'UserID' column
data_merged = data_scraped.dropna(subset=['UserID'])
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 475 entries, 0 to 623
Data columns (total 71 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   url                        475 non-null    object 
 1   title                      475 non-null    object 
 2   linkedinProfileUrl         437 non-null    object 
 3   email                      475 non-null    object 
 4   linkedinProfile            437 non-null    object 
 5   description                350 non-null    object 
 6   headline                   475 non-null    object 
 7   location                   475 non-null    object 
 8   firstName                  475 non-null    object 
 9   lastName                   475 non-null    object 
 10  fullName                   475 non-null    object 
 11  connectionDegree           475 non-null    object 
 12  vmid                       475 non-null    object 
 13  userId                     475 non-null    float64
 14 

In [219]:
data_merged.sample(20)

,url,title,linkedinProfileUrl,email,linkedinProfile,description,headline,location,firstName,lastName,...,imgUrl,website,mail,connectedOn,error,phoneNumber,partialScreenshot,facebookUrl,UserID,Attendance
365,https://linkedin.com/in/wallybiz,"Wally L. – Product Manager, Global Supplier Te...",https://www.linkedin.com/in/wallyproductmanager/,,https://www.linkedin.com/in/wallyproductmanager/,"A results-driven Product Manager, specializing...",Product Manager | E-Commerce | Supply Chain Da...,"Berlin, Berlin, Germany",wally,l,...,https://media.licdn.com/dms/image/C5603AQHsqdf...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,697.0,1
146,https://linkedin.com/in/seyma-tuzer-514124115,seyma tuzer - KPMG IT Service GmbH,https://www.linkedin.com/in/seyma-tuzer-514124...,,https://www.linkedin.com/in/seyma-tuzer-514124...,NaN,Civil Engineer & Computer Engineer,"Berlin, Germany",seyma,tuzer,...,https://media.licdn.com/dms/image/C4E03AQHO-I0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603.0,1
515,https://linkedin.com/in/milana-magurina,Milana Magurina – Product Manager – Vitatracker,https://www.linkedin.com/in/milana-magurina/,,https://www.linkedin.com/in/milana-magurina/,I love to work with people and I consistently ...,Product Manager | Enhancing Customer Experienc...,"Berlin, Berlin, Germany",milana,magurina,...,https://media.licdn.com/dms/image/D4E03AQEY-Ns...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,444.0,0
212,https://linkedin.com/in/mariliisalgma,Mari-Liis Algma - Delivery Hero,https://www.linkedin.com/in/mariliisalgma/,mariliis.algma@gmail.com,https://www.linkedin.com/in/mariliisalgma/,Personal website: www.mariliisalgma.com\n\nExp...,Senior Product Designer at Delivery Hero • 9+ ...,"Berlin, Berlin, Germany",mari-liis,algma,...,https://media.licdn.com/dms/image/D4E03AQH3dx_...,mariliisalgma.com,mariliis.algma@gmail.com,"Sep 4, 2023",NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,396.0,0
428,https://linkedin.com/in/lauracamilasoto/de,Laura Soto - und Gründungsberater – Tom Spike,NaN,,NaN,"Meine Leidenschaft gilt der Innovation, Kreati...",Strategic and Service Design / Innovation Cons...,"Berlin, Berlin, Germany",laura,soto,...,https://media.licdn.com/dms/image/C4D03AQFR1cF...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,361.0,0
229,https://linkedin.com/in/danieleronca,Daniele Ronca – Data Science & AI eleven-week ...,https://www.linkedin.com/in/danieleronca/,danieleronca88@gmail.com,https://www.linkedin.com/in/danieleronca/,NaN,Fostering product management culture w/ Berlin...,Berlin Metropolitan Area,daniele,ronca,...,https://media.licdn.com/dms/image/D4E03AQH-t0Z...,linktr.ee/berlinproductmanagers,danieleronca88@gmail.com,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,160.0,0
377,https://linkedin.com/in/sakicansev,Saki Cansev - Berlin School of Business & Inno...,https://www.linkedin.com/in/sakicansev/,,https://www.linkedin.com/in/sakicansev/,"Economist to be, Ney(reed flute) player, class...",Mekteb-i mülkiye,"Berlin, Berlin, Germany",saki,cansev,...,https://media.licdn.com/dms/image/D4D03AQGL4xu...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,570.0,0
539,https://linkedin.com/in/klara-murzabekova,Klara Murzabekova - Playrix,https://www.linkedin.com/in/klara-murzabekova/,,https://www.linkedin.com/in/klara-murzabekova/,"""I help teams make games"" \n ...",Product Manager,"Berlin, Berlin, Germany",klara,murzabekova,...,https://media.licdn.com/dms/image/C4D03AQGQtSV...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,349.0,0
62,https://linkedin.com/in/daniel-schwarz92,Daniel Schwarz – Product Manager – sennder,https://www.linkedin.com/in/daniel-schwarz92/,,https://www.linkedin.com/in/daniel-schwarz92/,I'm an economist turned IT Product Manager wit...,Product Manager | sennder,"Berlin, Berlin, Germany",daniel,schwarz,...,https://media.licdn.com/dms/image/C4E03AQFogz5...,Na

In [220]:

columns_to_drop = ['url', 'title', 'linkedinProfileUrl', 'email', 'linkedinProfile', 'firstName', 'lastName',
               'fullName', 'connectionDegree', 'timestamp', 'subscribers', 'mutualConnectionsText', 'imgUrl', 'website', 'mail',
               'profileId', 'baseUrl', 'connectionDegree', 'vmid', 'userId', 'linkedinSalesNavigatorUrl', 'connectionsCount', 'connectionsUrl',
               'mutualConnectionsUrl','companyUrl','companyUrl2','schoolUrl','schoolUrl2','jobDateRange2',
               'connectedOn', 'phoneNumber', 'partialScreenshot', 'facebookUrl', 'website', 'error']

# Drop columns from the merged
data_merged.drop(columns=columns_to_drop, inplace=True)
data_merged.set_index('UserID', inplace=True)

/var/folders/5f/td3vxmq52mgbxcd36rlnh7wc0000gn/T/ipykernel_19215/2823466866.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merged.drop(columns=columns_to_drop, inplace=True)


In [221]:
data_merged

,description,headline,location,company,jobTitle,jobDescription,jobLocation,jobDateRange,jobDuration,company2,...,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,Attendance
UserID,,,,,,,,,,,,,,,,,,,,,
512.0,Experienced professional with the ability to c...,"General Manager - US Finance, Legal, HR, Accou...","Carrollton, Texas, United States",orbis systems,"General Manager, Orbis International Technolog...",A highly driven professional leading the US op...,"Carrollton, Texas, United States",May 2022 - Present,1 yr 10 mos,Texas Instruments,...,21.0,Strategic Planning,21.0,Forecasting,21.0,Organizational Development,21.0,Business Strategy,21.0,0
652.0,"I am a Product Owner at World Simplified UG, a...",Tech product support intern at Holidu,"Berlin, Berlin, Germany",holidu,Tech product support intern,NaN,"Munich, Bavaria, Germany · Hybrid",Oct 2023 - Present,5 mos,World Simplified UG,...,NaN,Team Leadership,NaN,Cross-functional Team Leadership,1.0,Project Planning,21.0,Business Development,NaN,0
618.0,Designer & Problem Solver | CS Student | Aimin...,Making Interfaces Sexier - UI/UX/Dev,"Berlin, Berlin, Germany",a.t.e.m. management gmbh,Web Designer,Skills: Web Design · Cascading Style Sheets (C...,"Berlin, Germany",Aug 2022 - Present,1 yr 7 mos,Freelance,...,NaN,JavaScript,2.0,Cascading Style Sheets (CSS),2.0,Front-End Development,1.0,Wireframing,1.0,0
96.0,NaN,Product & Growth,"Berlin, Berlin, Germany",clink,Chief Experience Officer,NaN,"Berlin, Germany",Sep 2023 - Present,6 mos,OptioPay Group (now Clink),...,40.0,Cross-functional Team Leadership,3.0,Business Development,33.0,Mobile Applications,10.0,Mobile Technology,35.0,0
194.0,As a product marketing manager at the JetBrain...,Product marketing manager at JetBrains | Mentor,"Berlin, Berlin, Germany",jetbrains,Product Marketing Manager,NaN,"Berlin, Germany",Oct 2021 - Present,2 yrs 5 mos,Product Marketing Alliance,...,8.0,Marketing Communications,8.0,Customer Insight,13.0,Market Analysis,12.0,Marketing Research,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229.0,As a Senior Product Manager with over 12+ year...,Product Manager @ KAYAK,"Berlin, Berlin, Germany",kayak,Product Manager,NaN,"Berlin, Germany · On-site",Nov 2023 - Present,5 mos,Sastaticket.pk,...,1.0,Customer Onboarding,1.0,Mentoring,1.0,Product Development,1.0,Analytical Skills,NaN,1
461.0,A lady with a previous marketing background an...,Junior Product Designer✨| UX/UI | Figma | Rese...,"Berlin, Berlin, Germany",velents,Junior Product Designer,Velents AI is a hiring platform that will help...,"Berlin, Germany · Remote",May 2023 - Present,11 mos,Brainnest,...,NaN,Prototyping,NaN,User Research,NaN,User Experience Design (UED),NaN,Design Systems,21.0,1
10.0,"A passionate Product Leader Focused on People,...",Product Leader & Mentor • Lead Product Manager...,"Berlin, Berlin, Germany",spryker,Lead Product Manager,Skills: Product Management · Team Leadership ·...,"Berlin, Germany · Remote",Aug 2022 - Present,1 yr 8 mos,Spryker,...,2.0,Problem Solving,41.0,Team Management,NaN,Technical Understanding,31.0,Product Road Mapping,3.0,1


In [188]:
data_events_series.sample(20)

,Contacts,# registered,# cancel,# attended,# strike count,Active,Name,Surname,1,2,3,4,5,6,7,8,9,10,11,12
570,mohitb199@gmail.com,0,0,0,0,False,Mohit,Bansal,0,0,0,0,0,0,0,0,0,0,0,0
902,NaN,0,0,0,0,False,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
19,h.attaran@gmail.com,1,1,2,-2,True,hadi,attaran,0,0,1,0,0,1,0,0,0,0,0,0
818,NaN,0,0,0,0,False,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
236,sahil478@gmail.com,0,0,1,-1,False,Sahil,Aggarwal,0,0,0,0,0,1,0,0,0,0,0,0
512,events@hugolofquist.com,1,0,0,1,False,Hugo,Lofquist,0,0,0,0,0,0,0,0,0,0,0,0
130,shayb25.de@gmail.com,1,0,1,0,True,Shay,B,0,0,1,0,0,0,0,0,0,0,0,0
186,ekaterina.firsova@yahoo.com,0,0,1,-1,False,Ekaterina,Firsova,0,0,0,0,0,1,0,0,0,0,0,0
47,jens-p.fischer@gmx.de,0,0,2,-2,True,Yvonne,Hoeke,0,2,0,0,0,0,0,0,0,0,0,0
613,mstamboroski@gmail.com,0,1,0,-1,False,Maycon,Stamboroski,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
data_events_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Contacts        746 non-null    object
 1   # registered    999 non-null    int64 
 2   # cancel        999 non-null    int64 
 3   # attended      999 non-null    int64 
 4   # strike count  999 non-null    int64 
 5   Active          999 non-null    bool  
 6   Name            746 non-null    object
 7   Surname         746 non-null    object
 8   1               999 non-null    int64 
 9   2               999 non-null    int64 
 10  3               999 non-null    int64 
 11  4               999 non-null    int64 
 12  5               999 non-null    int64 
 13  6               999 non-null    int64 
 14  7               999 non-null    int64 
 15  8               999 non-null    int64 
 16  9               999 non-null    int64 
 17  10              999 non-null    int64 
 18  11        

### Processing function for names

In [144]:
import unicodedata

def process_name(name):

    # replace NaN with an empty string
    if pd.isna(name):
        return ''
    # strip dots and spaces and get only the longest/first word
    words = name.lower().strip(" .").split()

    # Initialize variables to keep track of the longest word
    longest_word = ''
    longest_length = 0

    # Iterate through each word
    for word in words:
        # If the current word is longer than the longest word found so far, or if it's equal in length but appears earlier
        if len(word) > longest_length or (len(word) == longest_length and words.index(word) < words.index(longest_word)):
            longest_word = word
            longest_length = len(word)
    
    # Normalize the string to decomposed Unicode form
    normalized_s = unicodedata.normalize('NFD', longest_word)
    
    # Remove non-spacing marks (special characters)
    stripped_s = ''.join(c for c in normalized_s if unicodedata.category(c) != 'Mn')
    
    return stripped_s

In [122]:
process_name('Pöñnpp îkljhsfgsg')

'ikljhsfgsg'

In [145]:
data_scraped.firstName = data_scraped.firstName.apply(process_name)

In [146]:
data_scraped.lastName = data_scraped.lastName.apply(process_name)

In [147]:
data_scraped.fullName = data_scraped.firstName + ' ' + data_scraped.lastName
data_scraped.fullName

0             patrick beeker
1               teena kumari
2           mischa gawronski
3                 shuvam sah
4             ayman chalhoub
               ...          
499           lydia dietrich
500        busayomi omotosho
501    schultz-zehden angela
502          abhishek khatri
503             ramin amirov
Name: fullName, Length: 504, dtype: object

In [204]:
data_events_ppl["First Name"] = data_events_ppl["First Name"].apply(process_name)
data_events_ppl["Surname"] = data_events_ppl["Surname"].apply(process_name)

In [205]:
data_events_ppl["fullName"] = data_events_ppl["First Name"] + ' ' + data_events_ppl["Surname"]
data_events_ppl.fullName

0            patrick beeker
1              teena kumari
2                nesha siwi
3                shuvam sah
4            ayman chalhoub
               ...         
1062            max beitler
1063    georgiana ciobotaru
1064          manizeh dossa
1065           angela garay
1066         valentin klein
Name: fullName, Length: 1067, dtype: object

## Aggregate attendees by their name and give every one unique ID. 
Store just the number of events they've attended

In [206]:
data_events_ppl.sort_values(by='fullName')


,Event,First Name,Surname,Email,Attendee Status,Your Job Position,Choose your role,Choose your role.1,Seniority,How did you hear from us?,Company,Rain,Day,Location,fullName
120,2,abdelrahman,elfar,abdelrahman.alfar@gmail.com,Checked In,Engineering,NaN,NaN,NaN,LinkedIn,SAP,Yes,Tuesday,Berlin,abdelrahman elfar
764,5,abdelrahman,elfar,abdelrahman.alfar@gmail.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,Sap,No,Thursday,Berlin,abdelrahman elfar
208,1,abdo,wahba,abdelrahman.wahba@gmail.com,Attending,NaN,NaN,NaN,NaN,NaN,NaN,No,Thursday,Berlin,abdo wahba
828,5,abhishek,agarwal,aggyabhishek@gmail.com,Checked In,Founder / CXO,NaN,NaN,NaN,Word of Mouth / Referral,Appreciate Capital,No,Thursday,Berlin,abhishek agarwal
841,5,abhishek,khatri,abhishekkhatri@hotmail.com,Checked In,Other,NaN,NaN,NaN,LinkedIn,zalando,No,Thursday,Berlin,abhishek khatri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,4,zulfiya,malyshenko,zulfia@malyshenko.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,ex-Moss,No,Tuesday,Berlin,zulfiya malyshenko
973,6,zulfiya,malyshenko,zulfia@malyshenko.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,Looking for something new,Yes,Thursday,Berlin,zulfiya malyshenko
311,1,zulfiya,malyshenko,zulfia@malyshenko.com,Checked In,NaN,NaN,NaN,NaN,NaN,NaN,No,Thursday,Berlin,zulfiya malyshenko
747,5,zulfiya,malyshenko,zulfia@malyshenko.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,Looking for new opportinities,No,Thursday,Berlin,zulfiya malyshenko


In [207]:
data_events_ppl.iloc[321]


Event                                                   1
First Name                                       abhishek
Surname                                            wabale
Email                        abhishekwabale0712@gmail.com
Attendee Status                                 Attending
Your Job Position                                     NaN
Choose your role                          Working student
Choose your role.1                                    NaN
Seniority                                             NaN
How did you hear from us?                        LinkedIn
Company                                            Tenera
Rain                                                   No
Day                                              Thursday
Location                                           Berlin
fullName                                  abhishek wabale
Name: 321, dtype: object

In [208]:
data_events_ppl.iloc[331]

Event                                                   1
First Name                                       abhishek
Surname                                            wabale
Email                        abhishekwabale0712@gmail.com
Attendee Status                                Checked In
Your Job Position                                     NaN
Choose your role                          Working Student
Choose your role.1                                    NaN
Seniority                                             NaN
How did you hear from us?                        LinkedIn
Company                                            Tenera
Rain                                                   No
Day                                              Thursday
Location                                           Berlin
fullName                                  abhishek wabale
Name: 331, dtype: object

In [210]:
data_events_ppl.iloc[45]

Event                                                 2
First Name                                     abolfazl
Surname                                  jafarzadehpour
Email                        a.jafarzadehpour@gmail.com
Attendee Status                               Attending
Your Job Position                               Product
Choose your role                        Product Manager
Choose your role.1                                  NaN
Seniority                                           NaN
How did you hear from us?                      LinkedIn
Company                                          Home24
Rain                                                Yes
Day                                             Tuesday
Location                                         Berlin
fullName                        abolfazl jafarzadehpour
Name: 45, dtype: object

In [211]:
data_events_ppl.iloc[580]

Event                                                 4
First Name                                     abolfazl
Surname                                  jafarzadehpour
Email                        a.jafarzadehpour@gmail.com
Attendee Status                              Checked In
Your Job Position                               Product
Choose your role                        Product Manager
Choose your role.1                                  NaN
Seniority                                           NaN
How did you hear from us?                      LinkedIn
Company                                          Home24
Rain                                                 No
Day                                             Tuesday
Location                                         Berlin
fullName                        abolfazl jafarzadehpour
Name: 580, dtype: object

In [150]:
# data_events_ppl.Event = data_events_ppl.Event.apply(lambda x: 0 if x == 0 else 1)

In [203]:
data_events_ppl.sample(15)


,Event,First Name,Surname,Email,Attendee Status,Your Job Position,Choose your role,Choose your role.1,Seniority,How did you hear from us?,Company,Rain,Day,Location
876,6,Shubha,Pallavi,shubhapallavi.eesl@gmail.com,Not Attending,NaN,NaN,NaN,NaN,NaN,ison energy,Yes,Thursday,Berlin
298,1,Daniele,Ronca,danieleronca88@gmail.com,Not Attending,NaN,Pm,NaN,NaN,NaN,Newton,No,Thursday,Berlin
205,1,Julia,Steier,hellgelbesonnenblume@gmail.com,Attending,NaN,NaN,NaN,NaN,NaN,NaN,No,Thursday,Berlin
932,6,Tauheed,Ahmed,tau123456789@gmail.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,Gloravel,Yes,Thursday,Berlin
283,1,Yuri,Orthey,yehtro@gmail.com,Attending,NaN,Product Manager,NaN,NaN,Word of Mouth / Referral,-,No,Thursday,Berlin
651,5,Christopher,Hahn,christopher.hahn98@icloud.com,Not Attending,NaN,NaN,NaN,NaN,NaN,Purpose,No,Thursday,Berlin
731,5,Sergey,Zadyraka,sergey.zadyraka@gmail.com,Not Attending,NaN,NaN,NaN,NaN,NaN,Adsquare GmbH,No,Thursday,Berlin
677,5,Fabius,Weiner,fabius.weiner@gmail.com,Not Attending,NaN,NaN,NaN,NaN,NaN,Heyjobs,No,Thursday,Berlin
329,1,Gaurav,G,ggulrajani@gmail.com,Checked In,NaN,NaN,NaN,NaN,NaN,NaN,No,Thursday,Berlin
642,5,Yasir,Qureshi,yasirqureshi111@gmail.com,Attending,Product,Product Manager,NaN,NaN,LinkedIn,Sumup,No,Thursday,Berlin


In [152]:
# Define a custom function to get the first non-null value
def first_non_null(series):
    return series.dropna().iloc[0] if not series.dropna().empty else None

In [189]:
unique_attendees = data_events_ppl.groupby(by='fullName', as_index=False)\
                                  .agg({'Event': 'count',
                                        'Email': 'first',
                                        'Company': 'first',
                                       'Choose your role':first_non_null,
                                       'Choose your role.1':first_non_null,
                                       'First Name':first_non_null,
                                       'Surname':first_non_null})
unique_attendees["numEvents"] = unique_attendees.Event

# Index in this table is now the user ID
unique_attendees['UserID'] = unique_attendees.index



In [161]:
unique_attendees.sample(15)

,fullName,Event,Email,Company,Choose your role,Choose your role.1,First Name,Surname,numEvents,UserID
437,michele cerullo,1,michele.cerullo@sensirion.com,Sensirion Connected Solutions,None,None,michele,cerullo,1,437
31,alex simukovic,3,alex.simukovic@adjust.com,Adjust,Product Manager,None,alex,simukovic,3,31
355,krutika gopalakrishnan,1,krutikagopalakrishnan@gmail.com,Mercedes pay,None,Marketing Associate,krutika,gopalakrishnan,1,355
376,luisa suda,3,lnsuda@gmail.com,Humanoo,Product Owner,None,luisa,suda,3,376
93,attia mo,3,atiamohamedein@gmail.com,Vestiaire Collective,Product Manager,None,attia,mo,3,93
539,ranjith naik,2,ranjith.innov@gmail.com,bbw,None,None,ranjith,naik,2,539
182,diana kryuchkova,1,d.kriuchkv@gmail.com,Distribusion,None,None,diana,kryuchkova,1,182
684,victor semencenco,1,victor.semencenco@gmail.com,Verkstedt,None,None,victor,semencenco,1,684
663,tom laufer,1,tom@getloops.ai,Loops,None,None,tom,laufer,1,663
530,radomir sebek,2,rasho.ieee@gmail.com,Company,None,None,radomir,sebek,2,530


In [191]:
unique_attendees.reset_index()

,index,fullName,Event,Email,Company,Choose your role,Choose your role.1,First Name,Surname,numEvents,UserID
0,0,abdelrahman elfar,2,abdelrahman.alfar@gmail.com,SAP,Product Manager,None,abdelrahman,elfar,2,0
1,1,abdo wahba,1,abdelrahman.wahba@gmail.com,None,None,None,abdo,wahba,1,1
2,2,abhishek agarwal,1,aggyabhishek@gmail.com,Appreciate Capital,None,None,abhishek,agarwal,1,2
3,3,abhishek khatri,1,abhishekkhatri@hotmail.com,zalando,None,None,abhishek,khatri,1,3
4,4,abhishek wabale,2,abhishekwabale0712@gmail.com,Tenera,Working student,None,abhishek,wabale,2,4
...,...,...,...,...,...,...,...,...,...,...,...
720,720,zimmermann jennifer,1,zimmermannjennifer43@gmail.com,Gsgsg,None,None,zimmermann,jennifer,1,720
721,721,ziyad mohiyudheen,2,mmziyad@gmail.com,Aviv Group,None,None,ziyad,mohiyudheen,2,721
722,722,zsolt pap,1,papjanoszsolt@yahoo.com,Axkro,eCommerce Manager,None,zsolt,pap,1,722
723,723,zsolt szrapko,1,szrapko.zsolt+eventbrite@gmail.com,GoTo,Product Manager,None,zsolt,szrapko,1,723


In [192]:
# we have numEvents now instead
unique_attendees = unique_attendees.drop(labels='Event',axis=1)
unique_attendees

,fullName,Email,Company,Choose your role,Choose your role.1,First Name,Surname,numEvents,UserID
0,abdelrahman elfar,abdelrahman.alfar@gmail.com,SAP,Product Manager,None,abdelrahman,elfar,2,0
1,abdo wahba,abdelrahman.wahba@gmail.com,None,None,None,abdo,wahba,1,1
2,abhishek agarwal,aggyabhishek@gmail.com,Appreciate Capital,None,None,abhishek,agarwal,1,2
3,abhishek khatri,abhishekkhatri@hotmail.com,zalando,None,None,abhishek,khatri,1,3
4,abhishek wabale,abhishekwabale0712@gmail.com,Tenera,Working student,None,abhishek,wabale,2,4
...,...,...,...,...,...,...,...,...,...
720,zimmermann jennifer,zimmermannjennifer43@gmail.com,Gsgsg,None,None,zimmermann,jennifer,1,720
721,ziyad mohiyudheen,mmziyad@gmail.com,Aviv Group,None,None,ziyad,mohiyudheen,2,721
722,zsolt pap,papjanoszsolt@yahoo.com,Axkro,eCommerce Manager,None,zsolt,pap,1,722
723,zsolt szrapko,szrapko.zsolt+eventbrite@gmail.com,GoTo,Product Manager,None,zsolt,szrapko,1,723


In [176]:
unique_attendees["Company"] = unique_attendees["Company"].apply(process_name)
unique_attendees

,fullName,Email,Company,Choose your role,Choose your role.1,First Name,Surname,numEvents,UserID
0,abdelrahman elfar,abdelrahman.alfar@gmail.com,sap,Product Manager,None,abdelrahman,elfar,2,0
1,abdo wahba,abdelrahman.wahba@gmail.com,,None,None,abdo,wahba,1,1
2,abhishek agarwal,aggyabhishek@gmail.com,appreciate,None,None,abhishek,agarwal,1,2
3,abhishek khatri,abhishekkhatri@hotmail.com,zalando,None,None,abhishek,khatri,1,3
4,abhishek wabale,abhishekwabale0712@gmail.com,tenera,Working student,None,abhishek,wabale,2,4
...,...,...,...,...,...,...,...,...,...
720,zimmermann jennifer,zimmermannjennifer43@gmail.com,gsgsg,None,None,zimmermann,jennifer,1,720
721,ziyad mohiyudheen,mmziyad@gmail.com,group,None,None,ziyad,mohiyudheen,2,721
722,zsolt pap,papjanoszsolt@yahoo.com,axkro,eCommerce Manager,None,zsolt,pap,1,722
723,zsolt szrapko,szrapko.zsolt+eventbrite@gmail.com,goto,Product Manager,None,zsolt,szrapko,1,723


### Add corresponding User ID by the name to the first two tables and then merge them

In [185]:
data_events_ppl.merge(unique_attendees[["UserID","fullName"]], how = 'right',on = "fullName")


,Event,First Name,Surname,Email,Attendee Status,Your Job Position,Choose your role,Choose your role.1,Seniority,How did you hear from us?,Company,Rain,Day,Location,fullName,UserID
0,1,abdelrahman,elfar,abdelrahman.alfar@gmail.com,Checked In,Engineering,NaN,NaN,NaN,LinkedIn,SAP,Yes,Tuesday,Berlin,abdelrahman elfar,0
1,1,abdelrahman,elfar,abdelrahman.alfar@gmail.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,Sap,No,Thursday,Berlin,abdelrahman elfar,0
2,1,abdo,wahba,abdelrahman.wahba@gmail.com,Attending,NaN,NaN,NaN,NaN,NaN,NaN,No,Thursday,Berlin,abdo wahba,1
3,1,abhishek,agarwal,aggyabhishek@gmail.com,Checked In,Founder / CXO,NaN,NaN,NaN,Word of Mouth / Referral,Appreciate Capital,No,Thursday,Berlin,abhishek agarwal,2
4,1,abhishek,khatri,abhishekkhatri@hotmail.com,Checked In,Other,NaN,NaN,NaN,LinkedIn,zalando,No,Thursday,Berlin,abhishek khatri,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,1,zulfiya,malyshenko,zulfia@malyshenko.com,Checked In,NaN,NaN,NaN,NaN,NaN,NaN,No,Thursday,Berlin,zulfiya malyshenko,724
1063,1,zulfiya,malyshenko,zulfia@malyshenko.com,Not Attending,NaN,NaN,NaN,NaN,NaN,ex-Moss,Yes,Thursday,Berlin,zulfiya malyshenko,724
1064,1,zulfiya,malyshenko,zulfia@malyshenko.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,ex-Moss,No,Tuesday,Berlin,zulfiya malyshenko,724
1065,1,zulfiya,malyshenko,zulfia@malyshenko.com,Checked In,Product,Product Manager,NaN,NaN,LinkedIn,Looking for new opportinities,No,Thursday,Berlin,zulfiya malyshenko,724


### Matching users with the data scraped from LinkedIn
It's a bit more challenging as the full name doesn't always match (only ~60% of cases), so we need a more sophisticated
matching algorithm

In [179]:
# Custom function to compare rows
def custom_user_matching(row_scraped, row_ids):
    # Compare if the full names match
    if row_scraped['fullName'] == row_ids['fullName']:
        return True

    # Compare the emails
    if row_scraped['email'] == row_ids['Email'] :
        return True

    if not pd.isna(row_scraped['company']):
        # Compare first or last name match AND the company match
        if (row_scraped['lastName'] in row_ids['Surname']) and ( not pd.isna(row_scraped['lastName'])) :
            if (row_ids['Company'] in row_scraped['company'].lower()) and ( len(row_ids['Company']) != 0):
                return True
        if (row_scraped['firstName'] in row_ids['First Name']) and ( not pd.isna(row_scraped['firstName'])) :
            if (row_ids['Company'] in row_scraped['company'].lower()) and ( len(row_ids['Company']) != 0):
                return True
            




In [180]:

data_scraped['UserID'] = float('nan')
# Iterate through each row in the ids DataFrame
for j, row_ids in unique_attendees.iterrows():
    # Iterate through each row in the scraped DataFrame
    for index_scraped, row_scraped in data_scraped.iterrows():
        # Check if the rows match using the custom matching function
        if custom_user_matching(row_scraped, row_ids):
            # If a match is found, add the userID from the ids DataFrame to the scraped DataFrame
            data_scraped.at[index_scraped, 'UserID'] = row_ids['UserID']
            break

data_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 70 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   url                        499 non-null    object 
 1   title                      499 non-null    object 
 2   linkedinProfileUrl         460 non-null    object 
 3   email                      138 non-null    object 
 4   linkedinProfile            460 non-null    object 
 5   description                350 non-null    object 
 6   headline                   488 non-null    object 
 7   location                   488 non-null    object 
 8   firstName                  504 non-null    object 
 9   lastName                   504 non-null    object 
 10  fullName                   504 non-null    object 
 11  connectionDegree           488 non-null    object 
 12  vmid                       488 non-null    object 
 13  userId                     488 non-null    float64

In [193]:
# Drop rows with NaN values in the 'UserID' column
data_merged = data_scraped.dropna(subset=['UserID'])

In [197]:
data_merged.set_index(keys=['UserID'],drop=False,inplace=True)

In [198]:
data_merged

,url,title,linkedinProfileUrl,email,linkedinProfile,description,headline,location,firstName,lastName,...,mutualConnectionsText,imgUrl,website,mail,connectedOn,error,phoneNumber,partialScreenshot,facebookUrl,UserID
UserID,,,,,,,,,,,,,,,,,,,,,
512.0,https://linkedin.com/in/patrick-beeker-4673311,Patrick Beeker,https://www.linkedin.com/in/patrick-beeker-467...,NaN,https://www.linkedin.com/in/patrick-beeker-467...,Experienced professional with the ability to c...,"General Manager - US Finance, Legal, HR, Accou...","Carrollton, Texas, United States",patrick,beeker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,512.0
652.0,https://linkedin.com/in/teena-kumari-0b8359271,Teena Kumari – Tech product support intern – H...,https://www.linkedin.com/in/teena-kumari-0b835...,NaN,https://www.linkedin.com/in/teena-kumari-0b835...,"I am a Product Owner at World Simplified UG, a...",Tech product support intern at Holidu,"Berlin, Berlin, Germany",teena,kumari,...,"Carlo Cantarini, Thomas Hartmann, and 28 other...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,652.0
618.0,https://linkedin.com/in/shuvam-kumar-sah-88590...,Shuvam Kumar Sah – Web Designer,https://www.linkedin.com/in/shuvam-kumar-sah-8...,NaN,https://www.linkedin.com/in/shuvam-kumar-sah-8...,Designer & Problem Solver | CS Student | Aimin...,Making Interfaces Sexier - UI/UX/Dev,"Berlin, Berlin, Germany",shuvam,sah,...,🤖 Leon Meier and Nisha Kumari are mutual conne...,https://media.licdn.com/dms/image/D4E03AQFcJ48...,dribbble.com/shuvam_sah,NaN,NaN,NaN,NaN,NaN,NaN,618.0
96.0,https://linkedin.com/in/chalhoub,Ayman Chalhoub - Clink,https://www.linkedin.com/in/chalhoub/,NaN,https://www.linkedin.com/in/chalhoub/,NaN,Product & Growth,"Berlin, Berlin, Germany",ayman,chalhoub,...,"Torben Schulz, Sonya Pobiedimska, and 2 other ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0
194.0,https://linkedin.com/in/berendeeva-elena,Elena Berendeeva - JetBrains,https://www.linkedin.com/in/berendeeva-elena/,elenaberendeeva@gmail.com,https://www.linkedin.com/in/berendeeva-elena/,As a product marketing manager at the JetBrain...,Product marketing manager at JetBrains | Mentor,"Berlin, Berlin, Germany",elena,berendeeva,...,"Igor Ranc, Vadym Kuzmenko 👨🏻‍💻, and 17 other m...",https://media.licdn.com/dms/image/D4E03AQHwIfc...,NaN,elenaberendeeva@gmail.com,"Jul 11, 2023",NaN,NaN,NaN,NaN,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337.0,https://linkedin.com/in/karlmfvillanueva,Karl Villanueva - Ostrom,https://www.linkedin.com/in/karlmfvillanueva/,NaN,https://www.linkedin.com/in/karlmfvillanueva/,"As the Co-Founder and CMO/CPO of Ostrom, we ar...",Co-Founder @ Ostrom ⚡️#EnergyJustBetter 🌍,"Berlin, Berlin, Germany",karl,villanueva,...,"John Ismailoglu, Jack Lancaster, and 111 other...",https://media.licdn.com/dms/image/D4E03AQEDYSl...,ostrom.de,NaN,"Feb 6, 2024",NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,337.0
596.0,https://linkedin.com/in/savani-nimonkar-3945911b2,Savani Nimonkar - GISMA Business School,https://www.linkedin.com/in/savani-nimonkar/,NaN,https://www.linkedin.com/in/savani-nimonkar/,Certified Project Manager with a robust backgr...,Certificated Project Manager | Expertise in Ou...,"Berlin, Berlin, Germany",savani,nimonkar,...,Ranjith Puttarama Sunanda Naik is a mutual con...,https://media.licdn.com/dms/image/D4D03AQFonJ8...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,596.0
378.0,https://linkedin.com/in/lydia-dietrich,Lydia Dietrich - Product Manager - GoStudent,https://www.linkedin.com/in/lydia-dietrich/,NaN,https://www.linkedin.com/in/lydia-dietrich/,NaN,Product Manager at GoStudent,"Berlin, Berlin, Germany",lydia,dietrich,...,Chiara Zanchetta and Maaz Bin Kashif are mutua...,https://media.licdn.com/dms/image/C4D03AQHU75_...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,378.0


In [195]:
data_scraped.merge(unique_attendees, on = "UserID")

,url,title,linkedinProfileUrl,email,linkedinProfile,description,headline,location,firstName,lastName,...,facebookUrl,UserID,fullName_y,Email,Company,Choose your role,Choose your role.1,First Name,Surname,numEvents
0,https://linkedin.com/in/patrick-beeker-4673311,Patrick Beeker,https://www.linkedin.com/in/patrick-beeker-467...,NaN,https://www.linkedin.com/in/patrick-beeker-467...,Experienced professional with the ability to c...,"General Manager - US Finance, Legal, HR, Accou...","Carrollton, Texas, United States",patrick,beeker,...,NaN,512.0,patrick beeker,patrick.beeker@gmail.com,Smartbroker AG,Product Manager,None,patrick,beeker,2
1,https://linkedin.com/in/teena-kumari-0b8359271,Teena Kumari – Tech product support intern – H...,https://www.linkedin.com/in/teena-kumari-0b835...,NaN,https://www.linkedin.com/in/teena-kumari-0b835...,"I am a Product Owner at World Simplified UG, a...",Tech product support intern at Holidu,"Berlin, Berlin, Germany",teena,kumari,...,NaN,652.0,teena kumari,teena2768@gmail.com,World simplifed Ug,Product Manager,Product Manager,teena,kumari,2
2,https://linkedin.com/in/shuvam-kumar-sah-88590...,Shuvam Kumar Sah – Web Designer,https://www.linkedin.com/in/shuvam-kumar-sah-8...,NaN,https://www.linkedin.com/in/shuvam-kumar-sah-8...,Designer & Problem Solver | CS Student | Aimin...,Making Interfaces Sexier - UI/UX/Dev,"Berlin, Berlin, Germany",shuvam,sah,...,NaN,618.0,shuvam sah,shuvamshah98152@gmail.com,ATEM,None,None,shuvam,sah,1
3,https://linkedin.com/in/chalhoub,Ayman Chalhoub - Clink,https://www.linkedin.com/in/chalhoub/,NaN,https://www.linkedin.com/in/chalhoub/,NaN,Product & Growth,"Berlin, Berlin, Germany",ayman,chalhoub,...,NaN,96.0,ayman chalhoub,aymanc@gmail.com,Clink,None,None,ayman,chalhoub,2
4,https://linkedin.com/in/berendeeva-elena,Elena Berendeeva - JetBrains,https://www.linkedin.com/in/berendeeva-elena/,elenaberendeeva@gmail.com,https://www.linkedin.com/in/berendeeva-elena/,As a product marketing manager at the JetBrain...,Product marketing manager at JetBrains | Mentor,"Berlin, Berlin, Germany",elena,berendeeva,...,NaN,194.0,elena berendeeva,elenaberendeeva@gmail.com,JetBrains,None,Product Marketing,elena,berendeeva,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,https://linkedin.com/in/karlmfvillanueva,Karl Villanueva - Ostrom,https://www.linkedin.com/in/karlmfvillanueva/,NaN,https://www.linkedin.com/in/karlmfvillanueva/,"As the Co-Founder and CMO/CPO of Ostrom, we ar...",Co-Founder @ Ostrom ⚡️#EnergyJustBetter 🌍,"Berlin, Berlin, Germany",karl,villanueva,...,NaN,337.0,karl villanueva,karl@ostrom.de,Ostrom,None,None,karl,villanueva,2
388,https://linkedin.com/in/savani-nimonkar-3945911b2,Savani Nimonkar - GISMA Business School,https://www.linkedin.com/in/savani-nimonkar/,NaN,https://www.linkedin.com/in/savani-nimonkar/,Certified Project Manager with a robust backgr...,Certificated Project Manager | Expertise in Ou...,"Berlin, Berlin, Germany",savani,nimonkar,...,NaN,596.0,savani nimonkar,savani.workspace@gmail.com,GISMA Business School,None,None,savani,nimonkar,1
389,https://linkedin.com/in/lydia-dietrich,Lydia Dietrich - Product Manager - GoStudent,https://www.linkedin.com/in/lydia-dietrich/,NaN,https://www.linkedin.com/in/lydia-dietrich/,NaN,Product Manager at GoStudent,"Berlin, Berlin, Germany",lydia,dietrich,...,NaN,378.0,lydia dietrich,laa.dietrich@gmail.com,Gostudent,Product Manager,None,lydia,dietrich,1
390,https://linkedin.com/in/busayomi-omotosho-1ab4...,Busayomi Omotosho - Hexad GmbH,https://www.linkedin.com/in/busayomi-omotosho-...,busayomiomotosho10@gmail.com,https://www.linkedin.com/in/busayomi-omotosho-...,Hello there! I'm Busayomi. \n\nI have been a p...,Product management | B2B | B2C | Product manag...,"Berlin, Berlin, Germany",busayomi,omotosho,...,NaN,121.0,busayomi omotosho,busayomiomotosho10@gmail.com,Hexad Gmbh,Product Manager,None,busayomi,omotosho,2


In [74]:
data_scraped.merge(unique_attendees, how="left", on = "fullName")


,url,title,linkedinProfileUrl,email,linkedinProfile,description,headline,location,firstName,lastName,...,imgUrl,website,mail,connectedOn,error,phoneNumber,partialScreenshot,facebookUrl,numEvents,UserID
0,https://linkedin.com/in/patrick-beeker-4673311,Patrick Beeker,https://www.linkedin.com/in/patrick-beeker-467...,NaN,https://www.linkedin.com/in/patrick-beeker-467...,Experienced professional with the ability to c...,"General Manager - US Finance, Legal, HR, Accou...","Carrollton, Texas, United States",patrick,beeker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,519.0
1,https://linkedin.com/in/teena-kumari-0b8359271,Teena Kumari – Tech product support intern – H...,https://www.linkedin.com/in/teena-kumari-0b835...,NaN,https://www.linkedin.com/in/teena-kumari-0b835...,"I am a Product Owner at World Simplified UG, a...",Tech product support intern at Holidu,"Berlin, Berlin, Germany",teena,kumari,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,657.0
2,https://linkedin.com/in/mischagawronski,Mischa Gawronski – Creative Director – Freelance,https://www.linkedin.com/in/mischagawronski/,NaN,https://www.linkedin.com/in/mischagawronski/,Passionate and challenging influencer in strat...,IGP Network | Creative Director,"Berlin, Berlin, Germany",mischa,gawronski,...,https://media.licdn.com/dms/image/C5603AQHb9Mw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://linkedin.com/in/shuvam-kumar-sah-88590...,Shuvam Kumar Sah – Web Designer,https://www.linkedin.com/in/shuvam-kumar-sah-8...,NaN,https://www.linkedin.com/in/shuvam-kumar-sah-8...,Designer & Problem Solver | CS Student | Aimin...,Making Interfaces Sexier - UI/UX/Dev,"Berlin, Berlin, Germany",shuvam kumar,sah,...,https://media.licdn.com/dms/image/D4E03AQFcJ48...,dribbble.com/shuvam_sah,NaN,NaN,NaN,NaN,NaN,NaN,1.0,623.0
4,https://linkedin.com/in/chalhoub,Ayman Chalhoub - Clink,https://www.linkedin.com/in/chalhoub/,NaN,https://www.linkedin.com/in/chalhoub/,NaN,Product & Growth,"Berlin, Berlin, Germany",ayman,chalhoub,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,https://linkedin.com/in/lydia-dietrich,Lydia Dietrich - Product Manager - GoStudent,https://www.linkedin.com/in/lydia-dietrich/,NaN,https://www.linkedin.com/in/lydia-dietrich/,NaN,Product Manager at GoStudent,"Berlin, Berlin, Germany",lydia,dietrich,...,https://media.licdn.com/dms/image/C4D03AQHU75_...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,1.0,381.0
500,https://linkedin.com/in/busayomi-omotosho-1ab4...,Busayomi Omotosho - Hexad GmbH,https://www.linkedin.com/in/busayomi-omotosho-...,busayomiomotosho10@gmail.com,https://www.linkedin.com/in/busayomi-omotosho-...,Hello there! I'm Busayomi. \n\nI have been a p...,Product management | B2B | B2C | Product manag...,"Berlin, Berlin, Germany",busayomi,omotosho,...,https://media.licdn.com/dms/image/D4E03AQEdfdC...,NaN,busayomiomotosho10@gmail.com,"Nov 9, 2023",NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,2.0,121.0
501,https://linkedin.com/in/schultz-zehden-angela-...,Schultz-Zehden Angela - SUBMARINER Network for...,https://www.linkedin.com/in/schultz-zehden-ang...,NaN,https://www.linkedin.com/in/schultz-zehden-ang...,NaN,Unternehmensinhaber s.Pro GmbH,Berlin Metropolitan Area,schultz-zehden,angela,...,https://media.licdn.com/dms/image/C5603AQH7y4I...,sustainable-projects.eu,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,NaN,NaN
502,https://linkedin.com/in/abhishek-khatri-015255...,Abhishek Khatri – Zalando,NaN,NaN,NaN,PID-IT | SICHERES POPM | PSM | AGILER COACH | ...,"Senior Program Manager @ Zalando | PGD-IT, PRI...",Berlin Metropolitan Area,abhishek,khatri,...,https://media.licdn.com/dms/image/D4D03AQHwVsy...,NaN,NaN,NaN,NaN,NaN,https://phantombuster.s3.amazonaws.com/nitYe1s...,NaN,1.0,3.0


In [75]:
data_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 71 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   url                        499 non-null    object 
 1   title                      499 non-null    object 
 2   linkedinProfileUrl         460 non-null    object 
 3   email                      138 non-null    object 
 4   linkedinProfile            460 non-null    object 
 5   description                350 non-null    object 
 6   headline                   488 non-null    object 
 7   location                   488 non-null    object 
 8   firstName                  488 non-null    object 
 9   lastName                   488 non-null    object 
 10  fullName                   488 non-null    object 
 11  connectionDegree           488 non-null    object 
 12  vmid                       488 non-null    object 
 13  userId                     488 non-null    float64

In [165]:
data_scraped.company

0                                Orbis Systems
1                                       Holidu
2                                    Freelance
3                     A.T.E.M. Management GmbH
4                                        Clink
                        ...                   
499                                  GoStudent
500                                 Hexad GmbH
501    SUBMARINER Network for Blue Growth EEIG
502                                    Zalando
503                              mytheresa.com
Name: company, Length: 504, dtype: object